<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант 16 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----
Описание задачи:
Создать базовый класс PaymentMethod в C#, который будет представлять
различные способы оплаты. На основе этого класса разработать 2-3 производных
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из
классов должны быть реализованы новые атрибуты и методы, а также
переопределены некоторые методы базового класса для демонстрации
полиморфизма.

Требования к базовому классу PaymentMethod:

• Атрибуты: ID способа оплаты (PaymentMethodId), Название способа оплаты
(MethodName), Минимальная сумма (MinAmount).

• Методы:
o ProcessPayment(decimal amount): метод для обработки платежа
указанной суммы.
o CheckMinimumAmount(decimal amount): метод для проверки
минимальной суммы платежа.
o GetPaymentDetails(): метод для получения деталей способа оплаты.

Требования к производным классам:
1. ОнлайнОплата (OnlinePayment): Должен содержать дополнительные
атрибуты, такие как URL платежной системы (PaymentUrl).
Метод ProcessPayment() должен быть переопределен для включения URL
платежной системы в процесс оплаты.
2. БанковскийПеревод (BankTransfer): Должен содержать дополнительные
атрибуты, такие как Банковские данные (BankData).
Метод CheckMinimumAmount() должен быть переопределен для проверки
минимальной суммы платежа с учетом банковских комиссий.
3. Наличные (CashPayment) (если требуется третий класс): Должен содержать
дополнительные атрибуты, такие как Место выдачи наличных
(CashPickupPoint). Метод GetPaymentDetails() должен быть переопределен
для отображения места выдачи наличных.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

// Базовый класс PaymentMethod
public class PaymentMethod
{
    private int _paymentMethodId;
    private string _methodName;
    private decimal _minAmount;
    private DateTime _paymentDate;
    private string _description;
    private bool _isCompleted;
    private string _currency;
    private List<PaymentHistory> _paymentHistory; // Добавлен атрибут PaymentHistory

    public int PaymentMethodId 
    { 
        get { return _paymentMethodId; } 
        set { _paymentMethodId = value; } 
    }

    public string MethodName 
    { 
        get { return _methodName; } 
        set { _methodName = value; } 
    }

    public decimal MinAmount 
    { 
        get { return _minAmount; } 
        set { _minAmount = value; } 
    }

    public DateTime PaymentDate
    {
        get { return _paymentDate; }
        set { _paymentDate = value; }
    }

    public string Description
    {
        get { return _description; }
        set { _description = value; }
    }

    public bool IsCompleted
    {
        get { return _isCompleted; }
        set { _isCompleted = value; }
    }

    public string Currency 
    {
        get { return _currency; }
        set { _currency = value; }
    }

    public List<PaymentHistory> PaymentHistory // Свойство PaymentHistory
    {
        get { return _paymentHistory; }
        set { _paymentHistory = value; }
    }

    public PaymentMethod(int paymentMethodId, string methodName, decimal minAmount, string currency)
    {
        PaymentMethodId = paymentMethodId;
        MethodName = methodName;
        MinAmount = minAmount;
        Currency = currency;
        PaymentHistory = new List<PaymentHistory>(); // Инициализация PaymentHistory
    }

    // Метод для обработки платежа
    public virtual void ProcessPayment(decimal amount)
    {
        // Логика обработки платежа, не вывод на консоль
        Console.WriteLine($"Обработка платежа методом {MethodName} на сумму {amount} {Currency}");
        PaymentDate = DateTime.Now;
        IsCompleted = true;
        AddPaymentHistory(amount); // Добавление записи в PaymentHistory
    }

    // Метод для проверки минимальной суммы платежа
    public virtual bool CheckMinimumAmount(decimal amount)
    {
        if (amount < MinAmount)
        {
            Console.WriteLine($"Сумма платежа меньше минимальной ({MinAmount} {Currency})");
            return false;
        }
        return true;
    }
        // Метод для получения деталей способа оплаты
    public virtual string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}";
    }

    // Метод для получения истории платежей
    public virtual List<PaymentHistory> GetPaymentHistory()
    {
        // Логика получения истории платежей
        return PaymentHistory;
    }

    // Дополнительный метод для добавления записи в PaymentHistory
    private void AddPaymentHistory(decimal amount)
    {
        PaymentHistory.Add(new PaymentHistory
        {
            PaymentId = PaymentHistory.Count + 1,
            PaymentMethod = MethodName,
            Amount = amount,
            PaymentDate = PaymentDate
        });
    }
}

// Производный класс ОнлайнОплата
public class OnlinePayment : PaymentMethod
{
    private string _paymentUrl;
    private string _transactionId;
    private int _attemptsCount; // Добавлен атрибут AttemptsCount

    public string PaymentUrl 
    { 
        get { return _paymentUrl; } 
        set { _paymentUrl = value; } 
    }

    public string TransactionId 
    { 
        get { return _transactionId; } 
        set { _transactionId = value; } 
    }

    public int AttemptsCount // Свойство AttemptsCount
    {
        get { return _attemptsCount; }
        set { _attemptsCount = value; }
    }

    public OnlinePayment(int paymentMethodId, string methodName, decimal minAmount, string paymentUrl, string currency) 
        : base(paymentMethodId, methodName, minAmount, currency)
    {
        PaymentUrl = paymentUrl;
        AttemptsCount = 0; // Инициализация AttemptsCount
    }

    // Переопределенный метод ProcessPayment()
    public override void ProcessPayment(decimal amount)
    {
        base.ProcessPayment(amount);
        TransactionId = Guid.NewGuid().ToString(); // Генерация ID транзакции
        Console.WriteLine($"Перенаправление на платежную страницу: {PaymentUrl}");
        AttemptsCount++; // Увеличение AttemptsCount после каждой попытки
    }

    // Перегруженный метод GetPaymentDetails()
    public string GetPaymentDetails(bool showTransactionId = false)
    {
        if (showTransactionId)
        {
            return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}, ID транзакции: {TransactionId}";
        }
        else
        {
            return base.GetPaymentDetails();
        }
    }
}

// Производный класс БанковскийПеревод
public class BankTransfer : PaymentMethod
{
    private string _bankData;
    private decimal _bankFee;
    private string _bankName;
    private bool _isBankDataVerified; // Добавлен атрибут IsBankDataVerified

    public string BankData 
    { 
        get { return _bankData; } 
        set { _bankData = value; } 
    }

    public decimal BankFee 
    { 
        get { return _bankFee; }
        set { _bankFee = value; } 
    }

    public string BankName 
    { 
        get { return _bankName; } 
        set { _bankName = value; } 
    }

    public bool IsBankDataVerified // Свойство IsBankDataVerified
    {
        get { return _isBankDataVerified; }
        set { _isBankDataVerified = value; }
    }

    public BankTransfer(int paymentMethodId, string methodName, decimal minAmount, string bankData, decimal bankFee, string bankName, string currency)
        : base(paymentMethodId, methodName, minAmount, currency)
    {
        BankData = bankData;
        BankFee = bankFee;
        BankName = bankName;
        IsBankDataVerified = false; // Инициализация IsBankDataVerified
    }

    // Переопределенный метод CheckMinimumAmount()
    public override bool CheckMinimumAmount(decimal amount)
    {
        if (amount < MinAmount + BankFee)
        {
            Console.WriteLine($"Сумма платежа меньше минимальной ({MinAmount} + {BankFee} {Currency})");
            return false;
        }
        return true;
    }

    // Переопределенный метод GetPaymentDetails()
    public override string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}, Банковские данные: {BankData}, Банк: {BankName}, Комиссия: {BankFee}";
    }

    // Дополнительный метод для получения информации о банке
    public string GetBankInfo()
    {
        return $"Название банка: {BankName}, Банковские данные: {BankData}";
    }

    // Дополнительный метод для проверки банковских данных
    public void VerifyBankData()
    {
        // Логика проверки банковских данных
        Console.WriteLine("Проверка банковских данных...");
        IsBankDataVerified = true;
    }
}

// Производный класс Наличные
public class CashPayment : PaymentMethod
{
    private string _cashPickupPoint;
    private string _paymentReference;
    private bool _isPaymentRecorded; // Добавлен атрибут IsPaymentRecorded

    public string CashPickupPoint 
    { 
        get { return _cashPickupPoint; } 
        set { _cashPickupPoint = value; } 
    }

    public string PaymentReference 
    { 
        get { return _paymentReference; } 
        set { _paymentReference = value; } 
    }

    public bool IsPaymentRecorded // Свойство IsPaymentRecorded
    {
        get { return _isPaymentRecorded; }
        set { _isPaymentRecorded = value; }
    }

    public CashPayment(int paymentMethodId, string methodName, decimal minAmount, string cashPickupPoint, string currency)
        : base(paymentMethodId, methodName, minAmount, currency)
    {
        CashPickupPoint = cashPickupPoint;
        IsPaymentRecorded = false; // Инициализация IsPaymentRecorded
    }

    // Переопределенный метод GetPaymentDetails()
    public override string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}, Место выдачи наличных: {CashPickupPoint}";
    }

    // Перегруженный метод GetPaymentDetails()
    public string GetPaymentDetails(bool showReference = false)
    {
        if (showReference)
        {
            return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}, Место выдачи наличных: {CashPickupPoint}, Номер платежа: {PaymentReference}";
        }
        else
        {
            return base.GetPaymentDetails();
        }
    }

    // Дополнительный метод для записи платежа
    public void RecordPayment(string paymentReference)
    {
        PaymentReference = paymentReference;
        Console.WriteLine($"Платеж записан с номером: {paymentReference}");
        IsPaymentRecorded = true; // Установка IsPaymentRecorded в true
    }
}

// Дополнительный класс - PrepaidCard (сложное наследование)
public class PrepaidCard : OnlinePayment
{
    private string _cardNumber;
    private DateTime _cardExpiryDate;
    private bool _isCardVerified; // Добавлен атрибут IsCardVerified

    public string CardNumber
    {
        get { return _cardNumber; }
        set { _cardNumber = value; }
    }

    public DateTime CardExpiryDate 
    {
        get { return _cardExpiryDate; }
        set { _cardExpiryDate = value; }
    }

    public bool IsCardVerified // Свойство IsCardVerified
    {
        get { return _isCardVerified; }
        set { _isCardVerified = value; }
    }

    public PrepaidCard(int paymentMethodId, string methodName, decimal minAmount, string paymentUrl, string cardNumber, DateTime cardExpiryDate, string currency) 
        : base(paymentMethodId, methodName, minAmount, paymentUrl, currency)
    {
        CardNumber = cardNumber;
        CardExpiryDate = cardExpiryDate;
        IsCardVerified = false; // Инициализация IsCardVerified
    }

    // Переопределенный метод ProcessPayment()
    public override void ProcessPayment(decimal amount)
    {
        base.ProcessPayment(amount);
        Console.WriteLine($"Используется карта: {CardNumber}, Срок действия: {CardExpiryDate:dd/MM/yyyy}");
    }

    // Дополнительный метод для проверки срока действия карты
    public bool IsCardExpired()
    {
        return CardExpiryDate < DateTime.Now;
    }

    // Дополнительный метод для проверки карты
    public void VerifyCard()
    {
        // Логика проверки карты
        Console.WriteLine("Проверка карты...");
        IsCardVerified = true;
    }
}

// Интерфейс для множественного наследования
public interface ICommissionable
{
    decimal Commission { get; set; }
}

// Класс BankTransfer, реализующий интерфейс ICommissionable
public class BankTransferWithCommission : BankTransfer, ICommissionable
{
    public BankTransferWithCommission(int paymentMethodId, string methodName, decimal minAmount, string bankData, decimal bankFee, string bankName, string currency)
        : base(paymentMethodId, methodName, minAmount, bankData, bankFee, bankName, currency)
    {
    }

    // Реализация свойства Commission из интерфейса
    public decimal Commission { get; set; }
}

// Generic класс для обработки платежей
public class PaymentProcessor<T> where T : PaymentMethod
{
    public void ProcessPayment(T paymentMethod)
    {
        if (paymentMethod.CheckMinimumAmount(100))
        {
            paymentMethod.ProcessPayment(100);

            if (paymentMethod is ICommissionable commissionablePaymentMethod)
            {
                Console.WriteLine($"Комиссия: {commissionablePaymentMethod.Commission}");
            }
        }
        else
        {
            Console.WriteLine("Сумма платежа меньше минимальной.");
        }
    }
}

// Класс для хранения истории платежей
public class PaymentHistory
{
    public int PaymentId { get; set; }
    public string PaymentMethod { get; set; }
    public decimal Amount { get; set; }
    public DateTime PaymentDate { get; set; }
}

// Создание объектов PaymentMethod
OnlinePayment onlinePayment = new OnlinePayment(1, "Онлайн оплата", 100, "https://example.payment.com", "RUB");
BankTransfer bankTransfer = new BankTransfer(2, "Банковский перевод", 25, "Счет: 0987654321", 10, "ПриватБанк", "RUB");
CashPayment cashPayment = new CashPayment(3, "Наличные", 50, "Офис компании", "RUB");
PrepaidCard prepaidCard = new PrepaidCard(4, "Оплата картой", 30, "https://example.payment.com", "00998877665544332211", DateTime.Now.AddYears(1), "RUB");
BankTransferWithCommission bankTransferWithCommission = new BankTransferWithCommission(5, "Банковский перевод с комиссией", 25, "Счет: 9876543210", 10, "Сбербанк", "RUB");
bankTransferWithCommission.Commission = 25;

// Создание объекта PaymentProcessor
PaymentProcessor<PaymentMethod> paymentProcessor = new PaymentProcessor<PaymentMethod>();

// Обработка платежей с помощью PaymentProcessor
paymentProcessor.ProcessPayment(onlinePayment);
paymentProcessor.ProcessPayment(bankTransfer);
paymentProcessor.ProcessPayment(cashPayment);
paymentProcessor.ProcessPayment(prepaidCard);
paymentProcessor.ProcessPayment(bankTransferWithCommission);

// Получение истории платежей 
List<PaymentHistory> paymentHistory = onlinePayment.GetPaymentHistory();

// Вывод истории платежей
Console.WriteLine("История платежей:");
foreach (PaymentHistory historyItem in paymentHistory)
{
    Console.WriteLine($"ID платежа: {historyItem.PaymentId}, Способ: {historyItem.PaymentMethod}, Сумма: {historyItem.Amount}, Дата: {historyItem.PaymentDate:dd/MM/yyyy}");
}

Обработка платежа методом Онлайн оплата на сумму 100 RUB
Перенаправление на платежную страницу: https://example.payment.com
Обработка платежа методом Банковский перевод на сумму 100 RUB
Обработка платежа методом Наличные на сумму 100 RUB
Обработка платежа методом Оплата картой на сумму 100 RUB
Перенаправление на платежную страницу: https://example.payment.com
Используется карта: 00998877665544332211, Срок действия: 04/11/2025
Обработка платежа методом Банковский перевод с комиссией на сумму 100 RUB
Комиссия: 25
История платежей:
ID платежа: 1, Способ: Онлайн оплата, Сумма: 100, Дата: 04/11/2024
